<a href="https://colab.research.google.com/github/hikaruyaku/KIKAGAKU-/blob/master/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E5%AE%9F%E8%A3%85(%E5%9F%BA%E7%A4%8E).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データセットの準備

In [0]:
from sklearn.datasets import load_iris

# Iris データセットの読み込み
x, t = load_iris(return_X_y = True)

In [3]:
# 形の確認
x.shape, t.shape

((150, 4), (150,))

In [4]:
# 型の確認
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [5]:
# データ型の確認
x.dtype, t.dtype

(dtype('float64'), dtype('int64'))

In [6]:
!pip install torch torchvision

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [9]:
#型の確認
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [10]:
#データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [11]:
#入力変数と目的変数をまとめて、ひとつのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x,t)
dataset

In [12]:
#型の確認
type(dataset)

torch.utils.data.dataset.TensorDataset

In [13]:
# (入力値、目標値)のようにタプルで収納されている。
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [14]:
# 型の確認
type(dataset[0])

tuple

In [15]:
# 1 サンプル目の入力値
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [16]:
# 1 サンプル目の目標値
dataset[0][1]

tensor(0)

In [17]:
# サンプル数は len で取得可能
len(dataset)

150

In [0]:
# 各データセットのサンプル数を決定
# train : val ; test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [19]:
# それぞれのサンプル数を確認
n_train, n_val, n_test

(90, 30, 30)

In [0]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [21]:
# サンプル数の確認
len(train), len(val), len(test)

(90, 30, 30)

## ミニバッチ学習

In [0]:
# バッチサイズ
batch_size = 10

In [0]:
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

## モデルの定義

In [0]:
class Net(nn.Module):


  # 使用するオブジェクトを定義
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(4,4) # 全結合層"fc1"input:4->output:4
    self.fc2 = nn.Linear(4,3) # 全結合層"fc2"input:4->output:3

  # 順伝播
  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    return x

In [0]:
# 乱数のシードを固定して再現性を確保
torch.manual_seed(0)

# インスタンス化
net = Net()

In [29]:
# モデルの確認
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

## 目的関数を選択

In [30]:
# 目的関数の設定
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy>

## 最適化手法の選択

In [31]:
# net.parameters()を展開
for parameter in iter(net.parameters()):
  print(parameter)

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)
